# Exam Project: The formation of ISIS' Social Media Network

Group members: Zeyu Zhao, Helge Zille, Edith Zink, Sina Smid

In [3]:
import os
import requests
import re
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
plt.style.use('ggplot')
import datetime

from our_functions import *

**Data import:** 
- Dataset is downloaded from Kaggle (https://www.kaggle.com/fifthtribe/how-isis-uses-twitter)
- Some of text is in Arabic, this will be ignored (encoding = 'Latin-1') and only Latin (English?) text will be used in the analysis

In [4]:
df = read_tweets('tweets_1.csv')
df

,name,username,description,location,followers,numberstatuses,time,tweets,date,translated
0,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:07:00,'A MESSAGE TO THE TRUTHFUL IN SYRIA - SHEIKH ...,2015-01-06,True
1,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:27:00,SHEIKH FATIH AL JAWLANI 'FOR THE PEOPLE OF IN...,2015-01-06,True
2,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:29:00,FIRST AUDIO MEETING WITH SHEIKH FATIH AL JAWL...,2015-01-06,True
3,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:37:00,"SHEIKH NASIR AL WUHAYSHI (HA), LEADER OF AQAP...",2015-01-06,True
4,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:45:00,AQAP: 'RESPONSE TO SHEIKH BAGHDADIS STATEMENT...,2015-01-06,True
5,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:51:00,THE SECOND CLIP IN A DA'WAH SERIES BY A SOLDIE...,2015-01-06,False
6,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:04:00,OH MURABIT! : http://t.co/hujLj9KGkG http://t...,2015-01-06,True
7,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:06:00,'A COLLECTION OF THE WORDS OF THE U'LAMA REGA...,2015-01-06,True
8,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:17:00,Aslm Please share our new account after the pr...,2015-01-06,False
9,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-10 00:05:00,AQAP STATEMENT REGARDING THE BLESSED RAID IN ...,2015-01-10,True


**Descriptive Analysis** moved to another notebook!

**Data cleaning:**
- Prepare `tweets` column for text analysis
- Tokenize in different ways 

**Tokenization**

**Method 1**: Twitter preprocessor (https://pypi.org/project/tweet-preprocessor/) - Don't forget: `pip install tweet-preprocessor`
- Preprocessor is a preprocessing library for tweet data written in Python.
- When building Machine Learning systems based on tweet data, a preprocessing is required. This library makes it easy to clean, parse or tokenize the tweets.
- Currently supports cleaning, tokenizing and parsing: URLs, Hashtags, Mentions, Reserved words (RT, FAV), Emojis, Smileys

In [7]:
# tweets is the uncleaned column in the dataset
import preprocessor as p


SyntaxError: invalid syntax (preprocessor.py, line 10)

**Tokenization**
- **Method 2**: We tokenize ourselves using word split 

In [5]:
# BEFORE WORD SPLIT

# TO-DO What is about the \n??
# How to reasonable delete non-word characters?

# Extract the @ tags and remove it from the tweets
def extract_from_to_column(df, regex, from_col, to_col):
    df[to_col]=df[from_col].apply(lambda x: " ".join(regex.findall(x)))
    return df

def remove_from_body(df,regex):
    df['tweets']=df.tweets.apply(lambda x: re.sub(regex,'',x))
    return df

regex1 = re.compile("@(\S+)")   #tagged users
regex2 = re.compile("http\S+")  # urls
regex3 = re.compile("ENGLISH TRANS[^:]*:") #prefix
regex4 = re.compile("#(\S+)") #hashtags

data = extract_from_to_column(data, regex1, 'tweets', 'tags')
data = remove_from_body(data, regex1)

data = extract_from_to_column(data, regex2, 'tweets', 'links')
data = remove_from_body(data, regex2)

data = remove_from_body(data, regex3)

data.tags = data.tags.str.split()

NameError: name 'data' is not defined

In [17]:
# Version 1 Tokenization: Tokenize sentences into words: split into words
data['tweets_rec'] = data.tweets.str.strip().str.split('[\W_]+')
data

,name,username,description,location,followers,numberstatuses,time,tweets,language,lang_val,translated,tags,links,tweets_rec
0,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:07:00,: 'A MESSAGE TO THE TRUTHFUL IN SYRIA - SHEIKH...,nl,-25.419356,True,[],,"[, A, MESSAGE, TO, THE, TRUTHFUL, IN, SYRIA, S..."
1,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:27:00,: SHEIKH FATIH AL JAWLANI 'FOR THE PEOPLE OF I...,pl,-15.531152,True,[],,"[, SHEIKH, FATIH, AL, JAWLANI, FOR, THE, PEOPL..."
2,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:29:00,: FIRST AUDIO MEETING WITH SHEIKH FATIH AL JAW...,de,-32.171995,True,[],,"[, FIRST, AUDIO, MEETING, WITH, SHEIKH, FATIH,..."
3,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:37:00,": SHEIKH NASIR AL WUHAYSHI (HA), LEADER OF AQA...",en,-11.085660,True,[],,"[, SHEIKH, NASIR, AL, WUHAYSHI, HA, LEADER, OF..."
4,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:45:00,: AQAP: 'RESPONSE TO SHEIKH BAGHDADIS STATEMEN...,en,9.061840,True,[],,"[, AQAP, RESPONSE, TO, SHEIKH, BAGHDADIS, STAT..."
5,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 21:51:00,THE SECOND CLIP IN A DA'WAH SERIES BY A SOLDIE...,pl,-21.223762,False,[],,"[THE, SECOND, CLIP, IN, A, DA, WAH, SERIES, BY..."
6,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:04:00,: OH MURABIT! :,en,-1.319836,True,[],,"[, OH, MURABIT, ]"
7,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:06:00,: 'A COLLECTION OF THE WORDS OF THE U'LAMA REG...,en,-0.862855,True,[],,"[, A, COLLECTION, OF, THE, WORDS, OF, THE, U, ..."
8,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-06 22:17:00,Aslm Please share our new account after the pr...,en,-250.258157,False,[],,"[Aslm, Please, share, our, new, account, after..."
9,GunsandCoffee,GunsandCoffee70,ENGLISH TRANSLATIONS: http://t.co/QLdJ0ftews,NaN,640,49,2015-01-10 00:05:00,: AQAP STATEMENT REGARDING THE BLESSED RAID IN...,en,9.061840,True,[],,"[, AQAP, STATEMENT, REGARDING, THE, BLESSED, R..."


In [10]:
# Break words into different rows, by username
rows = list()
for row in data[['username', 'tweets_rec']].iterrows():
    r = row[1]
    for word in r.tweets_rec:
        rows.append((r.username, word))

tweets_words = pd.DataFrame(rows, columns=['username', 'word'])
tweets_words.head(20)

,username,word
0,GunsandCoffee70,
1,GunsandCoffee70,A
2,GunsandCoffee70,MESSAGE
3,GunsandCoffee70,TO
4,GunsandCoffee70,THE
5,GunsandCoffee70,TRUTHFUL
6,GunsandCoffee70,IN
7,GunsandCoffee70,SYRIA
8,GunsandCoffee70,SHEIKH
9,GunsandCoffee70,ABU


In [11]:
# Removing empty cells
tweets_words = tweets_words[tweets_words.word.str.len() > 0]
tweets_words.head()

,username,word
1,GunsandCoffee70,A
2,GunsandCoffee70,MESSAGE
3,GunsandCoffee70,TO
4,GunsandCoffee70,THE
5,GunsandCoffee70,TRUTHFUL


In [12]:
# lower case words
tweets_words['word'] = tweets_words.word.str.lower()
tweets_words.head(100)
# Note: we still need to remove https and @, https is included as: 1st word) https, 2nd word) t, 3rd word) co (see below as example for row 20-22)

,username,word
1,GunsandCoffee70,a
2,GunsandCoffee70,message
3,GunsandCoffee70,to
4,GunsandCoffee70,the
5,GunsandCoffee70,truthful
6,GunsandCoffee70,in
7,GunsandCoffee70,syria
8,GunsandCoffee70,sheikh
9,GunsandCoffee70,abu
10,GunsandCoffee70,muhammed


In [13]:
# dropping https and t, co
tweets_words.drop(['https', 't', 'co']) 
tweets_words.head()

KeyError: "['https' 't' 'co'] not found in axis"

**Note:** In the next part, I played around with the data to see what is possible using a blogpost online: https://sigdelta.com/blog/text-analysis-in-pandas/, which uses a TF-IDF method

In [ ]:
counts = tweets_words.groupby('username')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})
counts.head()

In [ ]:
def pretty_plot_top_n(series, top_n=5, index_level=0):
    r = series\
    .groupby(level=index_level)\
    .nlargest(top_n)\
    .reset_index(level=index_level, drop=True)
    r.plot.bar()
    return r.to_frame()

pretty_plot_top_n(counts['n_w'])

In [ ]:
word_sum = counts.groupby(level=0)\
    .sum()\
    .rename(columns={'n_w': 'n_d'})
word_sum

In [ ]:
tf = counts.join(word_sum)

tf['tf'] = tf.n_w/tf.n_d

tf.head()

In [ ]:
pretty_plot_top_n(tf['tf'])

In [ ]:
c_d = tweets_words.username.nunique()
c_d

In [ ]:
idf = tweets_words.groupby('word')\
    .username\
    .nunique()\
    .to_frame()\
    .rename(columns={'username':'i_d'})\
    .sort_values('i_d')
idf.head()

In [ ]:
idf['idf'] = np.log(c_d/idf.i_d.values)

idf.head()

In [ ]:
tf_idf = tf.join(idf)

tf_idf.head()

In [ ]:
tf_idf['tf_idf'] = tf_idf.tf * tf_idf.idf
tf_idf.head()

In [ ]:
pretty_plot_top_n(tf_idf['tf_idf'])